# 🔧 API Testing Notebook - UPDATED for PORT 8000

## 📝 Endpoint Changes Summary:
- **Server Port**: Changed from `8000` to `8000`
- **Chat Endpoint**: `api/chat/send` → `api/legal-chat/send-query`
- **Query Analysis**: Merged into `api/legal-chat/send-query`
- **RAG Endpoints**: Updated to port `8000`
- **Payload Structure**: Updated to match new API requirements

## ✅ Current Working Endpoints:
- `/health` - Health check
- `/ready` - Readiness check  
- `/api/status/worker` - Celery worker status
- `/api/legal-chat/send-query` - Main chat endpoint
- `/api/rag/retrieve` - Document retrieval
- `/api/rag/web_search` - Web search

---

In [1]:
# 🚀 Quick Test All Main Endpoints
import requests

base_url = "http://localhost:8000"

def test_endpoint(name, method, url, payload=None):
    try:
        if method == "GET":
            response = requests.get(url, timeout=10)
        else:
            response = requests.post(url, json=payload, timeout=10)
        
        print(f"✅ {name}: {response.status_code}")
        if response.status_code == 200:
            result = response.json()
            if isinstance(result, dict) and 'task_id' in result:
                print(f"   📋 Task ID: {result['task_id'][:8]}...")
        else:
            print(f"   ⚠️ Status: {response.status_code}")
    except Exception as e:
        print(f"❌ {name}: Error - {str(e)}")

print("🔍 Testing Main Endpoints...")
print("-" * 40)

# Health endpoints
test_endpoint("Health Check", "GET", f"{base_url}/health")
test_endpoint("Ready Check", "GET", f"{base_url}/ready")
test_endpoint("Worker Status", "GET", f"{base_url}/api/status/worker")

# Main API endpoints
test_endpoint("Legal Chat", "POST", f"{base_url}/api/legal-chat/send-query", {
    "user_id": "test_user",
    "query": "Luật giao thông",
    "conversation_id": "test_conv"
})

test_endpoint("RAG Retrieve", "POST", f"{base_url}/api/rag/retrieve", {
    "query": "luật hôn nhân"
})

test_endpoint("Web Search", "POST", f"{base_url}/api/rag/web_search", {
    "query": "luật lao động"
})

print("-" * 40)
print("🏁 Quick test completed!")

🔍 Testing Main Endpoints...
----------------------------------------
✅ Health Check: 200
✅ Ready Check: 200
✅ Worker Status: 200
✅ Legal Chat: 422
   ⚠️ Status: 422
✅ RAG Retrieve: 200
   📋 Task ID: 03c9e39b...
✅ Web Search: 200
   📋 Task ID: 28d5cb24...
----------------------------------------
🏁 Quick test completed!


# Test api send

In [9]:
import requests

# UPDATED ENDPOINT - Using port 8000 and new legal-chat endpoint
url = "http://localhost:8000/api/legal-chat/send-query"

# OLD ENDPOINT (commented out)
# url = "http://localhost:8000/api/chat/send"

payload = {
    "user_id": "tinh123",
    "message": "Tôi muốn tìm hiểu về luật hôn nhân và gia đình",
    "conversation_id": "test_api"
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers, timeout=30)

print("Status Code:", response.status_code)
print("Response JSON:", response.json())

Status Code: 200
Response JSON: {'conversation_id': 'test_api', 'status': 'processing', 'message': 'Đang phân tích câu hỏi pháp luật của bạn...', 'estimated_time': '30-60 giây'}


# Test api retrieve

In [29]:
import requests
import time
import redis

# UPDATED ENDPOINT - Using port 8000 
url = "http://localhost:8000/api/rag/retrieve"

# OLD ENDPOINT (commented out)
# url = "http://localhost:8000/api/rag/retrieve"

payload = {
    "query": "trốn nghĩa vụ quân sự",
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("Kết quả truy xuất:")
    print(data)
else:
    print(f"Lỗi khi gọi API: {response.status_code} - {response.text}")

Kết quả truy xuất:
{'message': 'Retrieval task started', 'task_id': 'e4778be7-b8f5-405f-91f9-f8868d843ea9'}


In [23]:
time.sleep(3) # chờ xử lý chunks

In [9]:
import json
import redis

In [ ]:
host = "localhost:6379"
redis_client = redis.from_url("redis://{}/2".format(host))
raw_chunks = redis_client.lrange("retrieval_processed_chunks", 0, 10)
chunks = [json.loads(chunk) for chunk in raw_chunks]
chunks

# test api query analyze

In [37]:
import requests

# UPDATED ENDPOINT - Using port 8000 and new legal-chat structure
url = "http://localhost:8000/api/legal-chat/analyze-legal-query"

# OLD ENDPOINT (commented out)
# url = "http://localhost:8000/api/chat/query/analyze"

payload = {
    "conversation_id": "6896ffe02304b4bdd7042847",
    "user_id": "tinh123",
    "query": "thông tin về luật hôn nhân và gia đình"
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("Kết quả truy xuất:")
    print(data)
else:
    print(f"Lỗi khi gọi API: {response.status_code} - {response.text}")

Kết quả truy xuất:
{'analysis': {'rewritten_query': 'luật hôn nhân và gia đình', 'query_type': 'single', 'reasoning': 'Câu hỏi liên quan đến luật hôn nhân và gia đình cần cập nhật thông tin mới nhất từ internet và dữ liệu pháp luật.'}, 'actions': [{'tool': 'web_search', 'input': 'luật hôn nhân và gia đình Việt Nam 2025', 'reason': 'Thông tin cần tham khảo thêm để cập nhật mới nhất.'}, {'tool': 'laws_retrieval', 'input': 'luật hôn nhân và gia đình', 'reason': 'Thông tin có thể tìm thấy trong kho dữ liệu pháp luật.'}], 'final_answer': 'Luật hôn nhân và gia đình quy định về các vấn đề liên quan đến hôn nhân, ly hôn, quyền và nghĩa vụ của các bên trong gia đình, cũng như các quy định pháp lý liên quan.'}


# test api generate_response

In [13]:
import requests
import time
import httpx
import json

def listen_legal_response(conversation_id):
    # ✅ Sửa endpoint: conversation_id là query parameter, không phải path parameter
    url = f"http://localhost:8000/api/legal-chat/stream-legal-response?conversation_id={conversation_id}"

    with httpx.stream("GET", url, timeout=None) as response:
        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            return

        for chunk in response.iter_text():
            if not chunk:
                continue

            # Nếu server gửi [DONE] thì dừng
            if chunk.strip() == "[DONE]":
                print("\n[Completed]")
                break

            if chunk.startswith("[ERROR]"):
                print("\n[Error]", chunk)
                break

            # In trực tiếp token mới
            print(chunk, end="", flush=True)

In [14]:


# UPDATED ENDPOINTS - Using port 8000
url_retrieval = "http://localhost:8000/api/rag/retrieve"
url_web_search = "http://localhost:8000/api/rag/web_search"
url_generation = "http://localhost:8000/api/legal-chat/generate-legal-response"

# OLD ENDPOINTS (commented out)
# url_retrieval = "http://localhost:8000/api/rag/retrieve"
# url_web_search = "http://localhost:8000/api/rag/web_search"
# url_generation = "http://localhost:8000/api/chat/generate_response"

payload = {
    "query": "trốn nghĩa vụ quân sự",
}

headers = {
    "Content-Type": "application/json"
}

start_time = time.time()
# response = requests.post(url_retrieval, json=payload, headers=headers)
# print("Retrieval Response:", response.json(), "Time taken:", time.time() - start_time)

payload = {"query": "thông tin về trốn nghĩa vụ quân sự"}
# response = requests.post(url_web_search, json=payload, headers=headers)
# print("Web Search Response:", response.json(), "Time taken:", time.time() - start_time)

# Updated payload structure for new legal-chat endpoint


payload = {
    "conversation_id": "e74937ce-084e-49fd-aeb9-ed47a6fe9a4f",
    "user_id": "tinh123",
    "rewrite_query": "trốn nghĩa vụ quân sự",
    "use_web_search": True,
    "use_retrieval": False
}
url_generation = "http://localhost:8000/api/legal-chat/generate-legal-response"
response = requests.post(url_generation, json=payload, headers=headers)
# print("Generation Response:", response.json(), "Time taken:", time.time() - start_time)
# listen_legal_response("e74937ce-084e-49fd-aeb9-ed47a6fe9a4f")
response.json()  # Return the response JSON for further processing or testing
listen_legal_response("e74937ce-084e-49fd-aeb9-ed47a6fe9a4f")

### Phân tích và tư vấn pháp luật về trốn nghĩa vụ quân sự

#### 1. Các quy định pháp luật hiện hành

Theo **Bộ luật Hình sự năm 2015** (sửa đổi, bổ sung năm 2017), hành vi trốn nghĩa vụ quân sự được quy định tại **Điều 332**. Cụ thể:

- **Hành vi trốn nghĩa vụ quân sự**: Là hành vi không thực hiện nghĩa vụ quân sự khi đã được triệu tập hoặc có nghĩa vụ tham gia quân đội.
- **Hình phạt**: Người nào trốn nghĩa vụ quân sự có thể bị xử lý bằng các hình thức như:
  - Phạt tiền.
  - Cải tạo không giam giữ.
  - Phạt tù từ 6 tháng đến 3 năm, tùy thuộc vào mức độ vi phạm và các tình tiết giảm nhẹ hoặc tăng nặng.

Ngoài ra, **Luật Nghĩa vụ quân sự** năm 2015 cũng quy định rõ quyền và nghĩa vụ của công dân trong việc thực hiện nghĩa vụ quân sự, bao gồm cả các trường hợp được hoãn hoặc miễn nghĩa vụ. Các trường hợp hoãn hoặc miễn nghĩa vụ quân sự được quy định cụ thể, nhằm tạo điều kiện cho những công dân có lý do chính đáng.

#### 2. Thông tin cập nhật từ web

Trong những năm gần đây, Chính phủ 

In [44]:
import sys
sys.path.append("D:/Data/Legal-Retrieval/src")
from app.config.database import messages_col, redis_client_web, redis_client_retrieval


2025-08-14 16:39:54,463 - app.celery_config - INFO - 🔄 Celery configured with broker: redis://localhost:6379/0
2025-08-14 16:39:54,464 - app.celery_config - INFO - 📊 Task queues: ['rag_queue', 'embed_queue', 'retrival_queue', 'link_extract_queue']
2025-08-14 16:39:55,192 - app.config.database - INFO - Connecting to MongoDB: mongodb://localhost:27017
2025-08-14 16:39:55,206 - app.config.database - INFO - Connecting to async MongoDB: mongodb://localhost:27017
2025-08-14 16:39:55,208 - app.config.database - INFO - Connecting to Redis DB 0: redis://localhost:6379/0
2025-08-14 16:39:55,210 - app.config.database - INFO - Connecting to Redis DB 1: redis://localhost:6379/1
2025-08-14 16:39:55,211 - app.config.database - INFO - Connecting to Redis DB 2: redis://localhost:6379/2
2025-08-14 16:39:55,211 - app.config.database - INFO - Connecting to Qdrant: http://localhost:6333
2025-08-14 16:39:55,575 - httpx - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"


c:\Users\Admin\miniconda3\envs\crypto_agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-08-14 16:40:00,551 - datasets - INFO - PyTorch version 2.7.1 available.


Fetching 30 files: 100%|██████████| 30/30 [00:00<?, ?it/s]


2025-08-14 16:40:03,883 - FlagEmbedding.finetune.embedder.encoder_only.m3.runner - INFO - loading existing colbert_linear and sparse_linear---------
2025-08-14 16:40:03,890 - httpx - INFO - HTTP Request: GET http://localhost:6333/collections/chat_questions/exists "HTTP/1.1 200 OK"
2025-08-14 16:40:03,915 - httpx - INFO - HTTP Request: GET http://localhost:6333/collections/law_corpus_bge_v3/exists "HTTP/1.1 200 OK"
2025-08-14 16:40:04,460 - httpx - INFO - HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
{"timestamp": "2025-08-14T09:40:05.547930Z", "level": "INFO", "logger": "application", "message": "FastAPI application started", "module": "logging_config", "function": "info", "line": 136}


In [ ]:
redis_client_retrieval

<redis.client.Redis(<redis.connection.ConnectionPool(<redis.connection.Connection(host=localhost,port=6379,db=2)>)>)>

# Test celery

In [5]:
import sys
import os
sys.path.append(os.path.abspath("../../legal-retrieval"))

from tasks_embedding import process_and_embed_url

task_result = process_and_embed_url.apply_async(
    args=["https://example.com"],
    queue='embedding_queue'
)

print(f"Đã gửi task vào queue 'embedding_queue'. Task ID: {task_result.id}")

Đã gửi task vào queue 'embedding_queue'. Task ID: 3378dd5b-8bb8-43c4-911f-6fae321304b0


# Test api web_search


In [ ]:
# import sys
# import os
# sys.path.append(os.path.abspath("../../"))
import requests
# from app.config import api_client

In [30]:

# UPDATED ENDPOINT - Using port 8000
url = "http://localhost:8000/api/rag/web_search"

# OLD ENDPOINT (commented out)
# url = "http://localhost:8000/api/rag/web_search_v2"

payload = {
    "query": "thông tin về luật nghĩa vụ quân sự",
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("Kết quả truy xuất:")
    print(data)
else:
    print(f"Lỗi khi gọi API: {response.status_code} - {response.text}")

Kết quả truy xuất:
{'task_id': '8a78a5af-68ea-4950-a237-1269c6a00aa9', 'message': 'Link extraction task started successfully.'}


In [72]:
import time
import sys
sys.path.append("D:/Data/Legal-Retrieval")
start_time = time.time()
from app.web_search.extraction_service import extraction_service
print(time.time() - start_time)

0.0015077590942382812


# linh tinh

In [5]:
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
import openai
import json
from typing import Dict, Any

def analyze_user_query(query: str, model: str = "gpt-4") -> Dict[str, Any]:
    """
    Phân tích truy vấn người dùng và trả về các sub-query đã phân loại theo JSON format.
    """
    prompt = PROMPT_TEMPLATE.format(query=query.strip())
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            stream=True
        )
        content = response.choices[0].message.content.strip()

        # Làm sạch JSON
        # json_clean = clean_json_from_response(content)
        # result = json.loads(json_clean)
        return response

    except Exception as e:
        return {
            "error": str(e),
            "raw_response": content if 'content' in locals() else None
        }
    

In [33]:
user_input = "Mình đang muốn tìm hiểu về GPT-4 có giỏi suy luận không, với lại Claude 3 cũng được khen nhiều. Mô hình nào tốt hơn? À mà không biết có nên dùng API nào rẻ hơn nữa? dae ckse duien heh"
output = analyze_user_query(user_input)
print(output)

{'rewrite_query': 'GPT-4 có giỏi suy luận không và so với Claude 3 thì mô hình nào tốt hơn? Nên dùng API nào rẻ hơn?', 'use_web_search': True, 'rephrased_query': 'So sánh khả năng suy luận của GPT-4 và Claude 3. API nào rẻ hơn để sử dụng với các mô hình này?'}


In [ ]:
import httpx

print("Testing response from legal chat API...")

# UPDATED ENDPOINT - Using port 8000 and legal-chat
# Note: Streaming may not be available, using regular request
with httpx.Client(timeout=None) as client:
    response = client.post(
        "http://localhost:8000/api/legal-chat/send-query",
        json={
            "user_id": "test_user",
            "query": "Explain LLM trong ngữ cảnh luật pháp",
            "conversation_id": "test_conversation"
        },
    )
    print("Response:", response.json())

# OLD STREAMING ENDPOINT (commented out)
# with httpx.Client(timeout=None) as client:
#     with client.stream(
#         "POST",
#         "http://localhost:8000/api/chat/query/analyze",
#         json={"query": "Explain LLM"},
#     ) as response:
#         for chunk in response.iter_text():
#             print(chunk, end="", flush=True)

Testing streaming response from chat API...
Thought: Người dùng yêu cầu giải thích về "LLM" (Large Language Model). Tôi cần xác định rõ "LLM" là gì và cung cấp một giải thích rõ ràng, dễ hiểu. Vì đây là kiến thức chung, tôi có thể dựa trên kiến thức đã học để trả lời mà không cần dùng web_search.

Action: Bỏ qua web_search, vì tôi đã có kiến thức về "LLM".

Final Answer: LLM (Large Language Model) là một loại mô hình trí tuệ nhân tạo được huấn luyện trên lượng dữ liệu văn bản lớn để hiểu và sinh ra ngôn ngữ tự nhiên. Các mô hình này có khả năng thực hiện nhiều nhiệm vụ liên quan đến ngôn ngữ như dịch thuật, trả lời câu hỏi, viết văn, và phân tích cảm xúc. Một số ví dụ nổi bật của LLM bao gồm GPT của OpenAI, BERT của Google, và T5.

In [ ]:
import requests

print("Testing response from legal chat API...")

# UPDATED ENDPOINT - Using port 8000 and legal-chat
response = requests.post(
    "http://localhost:8000/api/legal-chat/send-query",
    json={
        "user_id": "test_user", 
        "query": "Explain LLM trong ngữ cảnh luật pháp",
        "conversation_id": "test_conversation"
    }
)

print("Response:", response.json())

# OLD STREAMING CODE (commented out)
# response = requests.post(
#     "http://localhost:8000/api/chat/query/analyze",
#     json={"query": "Explain LLM"},
#     stream=True
# )
# 
# for chunk in response.iter_text(chunk_size=1):
#     print(chunk, end="", flush=True)

Testing streaming response from chat API...


AttributeError: 'Response' object has no attribute 'iter_text'

# Test redis

In [ ]:
import redis
redis_client = redis.Redis(host='localhost', port=6379, db=, decode_responses=True)
redis_key =     "processed_chunks"

chunk_count = redis_client.llen(redis_key)

if chunk_count > 0:
    print(f"✅ Found {chunk_count} processed chunks")
    chunks = redis_client.lrange(redis_key, 0, -1)
    chunks = [json.loads(c) for c in chunks]
else:
    print("⛔ No chunks found for this task")

✅ Found 119 processed chunks


In [48]:
import redis
redis_client = redis.Redis(host='localhost', port=6379, db=2, decode_responses=True)
redis_key = "retrieval_chunks"

chunk_count = redis_client.llen(redis_key)

if chunk_count > 0:
    print(f"✅ Found {chunk_count} processed chunks")
    chunks = redis_client.lrange(redis_key, 0, -1)
    chunks = [json.loads(c) for c in chunks]
else:
    print("⛔ No chunks found for this task")

✅ Found 20 processed chunks


In [50]:
redis_client_retrieval.keys()

['retrieval_chunks']

In [51]:
redis_client_retrieval.get("retrieval_chunks")  

ResponseError: WRONGTYPE Operation against a key holding the wrong kind of value

In [ ]:
# import redis

# redis_client = redis.Redis(host='localhost', port=6379)
# redis_client.flushdb()  # Xóa toàn bộ dữ liệu trong Redis

True